In [1]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd
from keras import models, optimizers, backend
from keras.layers import core, convolutional, pooling
from sklearn import model_selection
import cv2
import random

from tensorflow.keras.utils import Sequence
import matplotlib.image as mpimg

import json
import matplotlib.pyplot as plt

from PIL import Image
import skimage 
from skimage.transform import resize

from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from sklearn.preprocessing import LabelEncoder


from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from skimage.io import imread
from lime import lime_image

from keras.applications import inception_v3 as inc_net
from keras.applications.imagenet_utils import decode_predictions




In [2]:
dataset = ('C:/Users/Ketan/Thesis/self-driving-car/challenges/challenge-2')

In [3]:
# Read the data
df = pd.io.parsers.read_csv(os.path.join(dataset, 'out.csv'))
# # Split data into training and validation sets
df_train, df_valid = model_selection.train_test_split(df, test_size=.2)

In [4]:
pd.options.mode.chained_assignment = None #default = warn

In [5]:
df_train['angle_range'] = pd.qcut(df_train['angle'], q=10)
df_valid['angle_range'] = pd.qcut(df_valid['angle'], q=10)

In [6]:
df_train['labels']= df_train['angle_range'].cat.codes
df_valid['labels']= df_valid['angle_range'].cat.codes

In [7]:
df_train['labels'] = df_train['labels'].apply(str)
df_valid['labels'] = df_valid['labels'].apply(str)


In [9]:
df_train.to_csv(''udacity_training_classification'')

NameError: name 'udacity_training_classification' is not defined

In [9]:
datagen=ImageDataGenerator(rescale=1./255)
train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory=dataset, x_col="File", y_col="labels", 
                                            class_mode="categorical", target_size=(480,640), batch_size=32)

Found 4491 validated image filenames belonging to 10 classes.


In [10]:
valid_generator=datagen.flow_from_dataframe(dataframe=df_valid, directory=dataset, 
                                            x_col="File", y_col="labels", class_mode="categorical", target_size=(480,640), 
                                            batch_size=32)

Found 1123 validated image filenames belonging to 10 classes.


In [11]:
model = models.Sequential()
model.add(convolutional.Convolution2D(16,3,3, input_shape=(480,640,3), activation='relu'))
model.add(pooling.MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(convolutional.Convolution2D(32, 3, 3, activation='relu'))
model.add(pooling.MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(convolutional.Convolution2D(64, 3, 3, activation='relu'))
model.add(pooling.MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(core.Flatten())
model.add(core.Dense(500, activation='relu'))
model.add(core.Dropout(.5))
model.add(core.Dense(100, activation='relu'))
model.add(core.Dropout(.25))
model.add(core.Dense(20, activation='relu'))
model.add(core.Dense(10, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-04), loss='categorical_crossentropy',metrics=['AUC', 'categorical_accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 160, 213, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 80, 107, 16)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 35, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 18, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 6, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 384)               0

In [13]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [14]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [15]:
checkpoint_filepath = './logs2'
callback1 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', verbose=0, 
                                               save_best_only=False, save_weights_only=False, mode='auto', 
                                               save_freq='epoch',options=None)

In [16]:
model.load_weights('udacity_classification_custom_model_v2.h5')

In [17]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20, callbacks=[callback, callback1])

C:\Users\Ketan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
140/140 [==============================] - 142s 966ms/step - loss: 0.6329 - auc: 0.9766 - categorical_accuracy: 0.7800 - val_loss: 0.5612 - val_auc: 0.9809 - val_categorical_accuracy: 0.8179
INFO:tensorflow:Assets written to: .\logs2\assets
Epoch 2/20
140/140 [==============================] - 131s 937ms/step - loss: 0.6148 - auc: 0.9780 - categorical_accuracy: 0.7914 - val_loss: 0.5685 - val_auc: 0.9800 - val_categorical_accuracy: 0.8295
INFO:tensorflow:Assets written to: .\logs2\assets
Epoch 3/20
140/140 [==============================] - 130s 928ms/step - loss: 0.5871 - auc: 0.9805 - categorical_accuracy: 0.7863 - val_loss: 0.5736 - val_auc: 0.9799 - val_categorical_accuracy: 0.8161
INFO:tensorflow:Assets written to: .\logs2\assets
Epoch 4/20
140/140 [==============================] - 131s 934ms/step - loss: 0.5679 - auc: 0.9811 - categorical_accuracy: 0.8042 - val_loss: 0.5633 - val_auc: 0.9802 - val_categorical_accuracy: 0.8152
INFO:tensorflow:Assets written to: .\logs2

In [19]:
model.save_weights('udacity_classification_custom_model_weights_v3.h5')

In [18]:
model.save('udacity_classification_custom_model_v3.h5')

In [25]:
from keras.applications import inception_v3 as inc_net


In [26]:
def transform_img_fn(path_list):
    out = []
    for img_path in path_list:
        img = image.load_img((img_path), target_size=(480,640))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = inc_net.preprocess_input(x)
        out.append(x)
    return np.vstack(out) /2 +0.5

In [27]:
images = transform_img_fn([os.path.join(r'C:/Users/Ketan/Thesis/self-driving-car/challenges/challenge-2/','1479425717280900950.jpg')])


In [28]:
model.predict(images)

array([[2.7272626e-04, 9.8335797e-01, 1.0109666e-02, 1.5271852e-03,
        1.8140408e-03, 1.4903632e-03, 1.4183132e-03, 4.0885689e-06,
        1.9641282e-06, 3.6494907e-06]], dtype=float32)

In [23]:
 def open_images(inference_folder: str) -> np.ndarray:
    """Loads images from a folder and prepare them for inferencing.

    Parameters
    ----------
    inference_folder : str
        Location of images for inferencing.

    Returns
    -------
    np.ndarray
        List of images as numpy arrays transformed to fit the efficient_net model input specs.
    """
    images = []
    for img in os.listdir(inference_folder):
        img_location = os.path.join(inference_folder, img)  # create full path to image

        with Image.open(img_location) as img:  # open image with pillow

            img = np.array(img)
            img = img[:, :, :3]
            img = np.expand_dims(img, axis=0)  # add 0 dimension to fit input shape of efficient_net
            img = inc_net.preprocess_input(img)

        images.append(img)
    images_array = np.vstack(images)  # combine images efficiently to a numpy array
    return images_array

In [24]:
images = open_images("C:/Users/Ketan/Thesis/self-driving-car/challenges/test_dataset1")


In [37]:
predictions = model.predict_generator(images)
images_names = os.listdir("C:/Users/Ketan/Thesis/self-driving-car/challenges/test_dataset")
for image_name, prediction in zip(images_names, predictions):
    print(image_name, prediction)

1479425719681268780.jpg [7.2221975e-09 1.7213597e-07 7.1565906e-04 9.4972956e-01 1.3229251e-02
 7.5207460e-03 1.7035704e-02 3.5625370e-04 2.8775318e-03 8.5350713e-03]
1479425719731264417.jpg [3.2595270e-07 7.3315715e-07 1.0422587e-03 7.5753844e-01 4.9150348e-02
 8.0104377e-03 3.6861550e-02 1.2549951e-02 7.0672423e-02 6.4173587e-02]
1479425719781262693.jpg [1.5169276e-07 7.9415749e-08 3.4231867e-04 1.8231438e-01 2.6431710e-02
 6.6739358e-03 2.9259533e-02 1.8395272e-01 4.7569504e-01 9.5330119e-02]
1479425719831276356.jpg [6.2930823e-08 5.6677759e-08 2.0123074e-04 3.8890791e-01 3.5094813e-02
 4.9493317e-03 3.6107324e-02 2.0522982e-02 1.7559575e-01 3.3862060e-01]
1479425719881419974.jpg [4.74698567e-07 3.33585376e-05 2.20506772e-04 3.23442295e-02
 3.44644208e-03 3.73651311e-02 1.02660134e-01 9.11220685e-02
 5.92144430e-01 1.40663296e-01]
1479425719931414543.jpg [8.1365602e-07 8.6704567e-05 2.4341630e-04 2.2599611e-02 1.4037301e-03
 2.3952996e-02 5.0344113e-02 3.7958350e-02 6.9601810e-01 1.